# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join(r"C:\Users\cchic\OneDrive\Documents\projects_challenges\OpenWeather-API-Analysis\citiesdatadf.csv")

cities_data_df = pd.read_csv(csv_file)
cities_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   570 non-null    int64  
 1   City ID      570 non-null    int64  
 2   City         570 non-null    object 
 3   Country      567 non-null    object 
 4   Longitude    570 non-null    float64
 5   Latitude     570 non-null    float64
 6   Temperature  570 non-null    float64
 7   Humidity     570 non-null    int64  
 8   Cloud        570 non-null    int64  
 9   Wind Speed   570 non-null    float64
dtypes: float64(4), int64(4), object(2)
memory usage: 44.7+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_data_df[["Latitude", "Longitude"]]
humidity = cities_data_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_criteria_df = cities_data_df[(cities_data_df["Temperature"] >= 70) & (cities_data_df["Temperature"] <= 80) & 
                                     (cities_data_df["Wind Speed"] < 10) & (cities_data_df["Cloud"] == 0)]
weather_criteria_df.head()



,Unnamed: 0,City ID,City,Country,Longitude,Latitude,Temperature,Humidity,Cloud,Wind Speed
132,132,3868633,Vallenar,CL,-70.7581,-28.5708,73.78,24,0,9.57
157,157,286621,Salalah,OM,54.0924,17.0151,78.89,78,0,3.44
224,224,172515,Abu Samrah,SY,37.1841,35.3029,71.06,43,0,8.23
277,277,4088628,Saraland,US,-88.0706,30.8207,76.95,42,0,2.64
447,447,1266849,Khatauli,IN,77.7167,29.2833,70.79,66,0,7.09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = weather_criteria_df
hotel_df["Hotel Name"]
hotel_df.head()

,Unnamed: 0,City ID,City,Country,Longitude,Latitude,Temperature,Humidity,Cloud,Wind Speed,Hotel Name
132,132,3868633,Vallenar,CL,-70.7581,-28.5708,73.78,24,0,9.57,Hotel Cecil
157,157,286621,Salalah,OM,54.0924,17.0151,78.89,78,0,3.44,Salalah Gardens Hotel
224,224,172515,Abu Samrah,SY,37.1841,35.3029,71.06,43,0,8.23,NaN
277,277,4088628,Saraland,US,-88.0706,30.8207,76.95,42,0,2.64,"Country Inn & Suites by Radisson, Saraland, AL"
447,447,1266849,Khatauli,IN,77.7167,29.2833,70.79,66,0,7.09,POP 82900 Royals Rana Hotel


In [7]:
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        params = {
            "keyword": "hotel", 
            "radius": 5000,
            "key": g_key,}
        
        lat = row["Latitude"]
        lon = row["Longitude"]
        
        params["location"] = f"{lat}, {lon}"
        
        hotels_geo = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotels_geo["results"][0]["name"]
        
    except IndexError:
            
            hotel_df.loc[index, "Hotel Name"] = "NaN"
            
hotel_df

C:\Users\cchic\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\cchic\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Unnamed: 0,City ID,City,Country,Longitude,Latitude,Temperature,Humidity,Cloud,Wind Speed,Hotel Name
132,132,3868633,Vallenar,CL,-70.7581,-28.5708,73.78,24,0,9.57,Hotel Cecil
157,157,286621,Salalah,OM,54.0924,17.0151,78.89,78,0,3.44,Salalah Gardens Hotel
224,224,172515,Abu Samrah,SY,37.1841,35.3029,71.06,43,0,8.23,NaN
277,277,4088628,Saraland,US,-88.0706,30.8207,76.95,42,0,2.64,"Country Inn & Suites by Radisson, Saraland, AL"
447,447,1266849,Khatauli,IN,77.7167,29.2833,70.79,66,0,7.09,POP 82900 Royals Rana Hotel
465,465,110690,Abhā,SA,42.5053,18.2164,70.05,12,0,3.44,Shafa Abha Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))